In [ ]:
from tensorflow.keras.regularizers import l1, l2, l1_l2

# Importing the required libraries
from src.utils import base_model_config, create_and_run_model, plot_graph, build_model, get_rca_data

In [ ]:
## Acquire and process data
X, y = get_rca_data()

In [ ]:
## Tuning layers in the network

accuracy_measures = {}
layer_list =[]
for layer_count in range(1,6):
    
    #32 nodes in each layer
    layer_list.append(32)
    
    model_config = base_model_config()
    X,Y = get_rca_data()
    
    model_config["HIDDEN_NODES"] = layer_list
    model_name = "Layers-" + str(layer_count)
    history=create_and_run_model(model_config,X,Y,model_name)
    
    accuracy_measures[model_name] = history.history["accuracy"]
    
plot_graph(accuracy_measures,"Accuracy vs Layers")

In [ ]:
# tuning the number of nodes in the network
accuracy_measures = {}

node_increment=8

for node_count in range(1,5):
    
    #have 2 hidden layers in the networks as selected above
    layer_list =[]
    for layer_count in range(2):
        layer_list.append(node_count * node_increment)
    
    model_config = base_model_config()
    X,Y = get_rca_data()
    
    model_config["HIDDEN_NODES"] = layer_list
    model_name = "Nodes-" + str(node_count * node_increment)
    history=create_and_run_model(model_config,X,Y, model_name)
    
    accuracy_measures[model_name] = history.history["accuracy"]

plot_graph(accuracy_measures,"Accuracy vs Nodes")

## Tuning backpropagation

In [ ]:
# use optimizers

accuracy_measures = {}

optimizer_list = ['sgd','rmsprop','adam','adagrad']
for optimizer in optimizer_list:
    
    model_config = base_model_config()
    X,Y = get_rca_data()
    
    model_config["OPTIMIZER"] = optimizer
    model_name = "Optimizer-" + optimizer
    history=create_and_run_model(model_config,X,Y, model_name)
    
    accuracy_measures[model_name] = history.history["accuracy"]
    
plot_graph(accuracy_measures,"Accuracy vs Optimizers")

In [ ]:
# tuning learning rate

accuracy_measures = {}

learning_rate_list = [0.001, 0.005,0.01,0.1,0.5]
for learning_rate in learning_rate_list:
    
    model_config = base_model_config()
    X,Y = get_rca_data()
    
    #Fix Optimizer to the one chosen above
    model_config["OPTIMIZER"]="rmsprop"
    model_config["LEARNING_RATE"] = learning_rate
    model_name="Learning-Rate-" + str(learning_rate)
    history=create_and_run_model(model_config,X,Y, model_name)
    
    #Using validation accuracy
    accuracy_measures[model_name] = history.history["accuracy"]
    
plot_graph(accuracy_measures,"Accuracy vs Learning Rate")

## Avoiding overfitting

In [ ]:
# tuning regularizers
accuracy_measures = {}

regularizer_list = [l1(0.01), l2(0.01), l1_l2(l1=0.01, l2=0.01)]
regularizer_names = ['l1', 'l2', 'l1_l2']
for regularizer, reg_name in zip(regularizer_list, regularizer_names):
    
    model_config = base_model_config()
    X,Y = get_rca_data()
    
    model_config["REGULARIZER"] = regularizer
    model_name = "Regularizer-" + str(reg_name)
    history=create_and_run_model(model_config,X,Y, model_name)
    
    #Switch to validation accuracy
    accuracy_measures[model_name] = history.history["val_accuracy"]

plot_graph(accuracy_measures,"Validation Accuracy vs Regularizers")

In [ ]:
# tuning dropout
accuracy_measures = {}

dropout_list = [0.0, 0.1, 0.2, 0.5]
for dropout in dropout_list:
    
    model_config = base_model_config()
    X,Y = get_rca_data()
    
    #Use the regularizer chosen above
    model_config["REGULARIZER"] = "l2"
    model_config["DROPOUT_RATE"] = dropout
    model_name="Dropout-" + str(dropout)
    history=create_and_run_model(model_config,X,Y, model_name)
    
    #Using validation accuracy
    accuracy_measures[model_name] = history.history["val_accuracy"]
    
plot_graph(accuracy_measures,"Validation Accuracy vs Dropout")

In [ ]:
from tensorflow.python.keras import callbacks
from tensorflow.keras import models as k_models
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Adagrad
from tensorflow.keras.regularizers import l1, l2, l1_l2
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split

def build_model(feature_shape=(7,),
                hidden_nodes=[32], 
                optimizer='adam',
                learning_rate=0.001, 
                dropout_rate=0.0,
                kernel_regularizer=None,
                normalization=None, 
                kernel_initializer=None, 
                bias_initializer=None):
    
    model = k_models.Sequential()
    model.add(Input(feature_shape))
    for nodes in hidden_nodes:
        model.add(Dense(nodes,
                        activation='relu',
                        kernel_regularizer=kernel_regularizer,
                        kernel_initializer=kernel_initializer,
                        bias_initializer=bias_initializer,
                        ))
        if normalization and normalization == "batch":
                model.add(BatchNormalization())
        if dropout_rate > 0.0:
            model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='linear'))

    if optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)
    elif optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'adagrad':
        opt = Adagrad(learning_rate=learning_rate)

    model.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])
    return model

# Acquire and process data
X, y = get_rca_data()

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, 
                                                    random_state=42)
# Wrap the model using KerasRegressor
dnn = KerasRegressor(model=build_model, verbose=1)
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


# Define the parameter grid
param_grid = {
    'model__hidden_nodes': [[16], [32, 32], [64, 64], [128, 128]],
    'model__optimizer': ['rmsprop', 'adam', 'sgd', 'adagrad'],
    'model__learning_rate': [0.001, 0.005, 0.01],
    'model__kernel_regularizer': [None, l1(0.01), l2(0.01), l1_l2(l1=0.01, l2=0.01)],
    'model__dropout_rate': [0.0, 0.2, 0.5],
    'model__normalization': [None, 'batch'],
    'epochs': [5, 10, 15, 25, 30],
    'model__bias_initializer': ['zeros', 'ones', 'glorot_uniform'],
    'model__kernel_initializer': ['glorot_uniform', 'he_normal', 'he_uniform']
}

rcv = RandomizedSearchCV(estimator=dnn,
                         param_distributions=param_grid,
                         scoring='neg_mean_absolute_error',
                         cv=5,
                         n_iter=10,
                         verbose=1)

history = rcv.fit(X_train, y_train, verbose=1, callbacks=[early_stopping])

# Print the best parameters and best score
print("Best: %f using %s" % (history.best_score_, history.best_params_)) 
# Best: -0.392004 
# Best params using {'model__optimizer': 'adam',
# 'model__normalization': None, 
# 'model__learning_rate': 0.01, 
# 'model__kernel_regularizer': L2, 
# 'model__kernel_initializer': 'he_normal',
# 'model__hidden_nodes': [64, 64],
# 'model__dropout_rate': 0.2,
# 'model__bias_initializer': 'zeros',
# 'epochs': 15}

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import numpy as np

y_pred = history.predict(X_test) # values is float, need to make it integer type by ceil method
y_pred = np.round(y_pred)
acc = accuracy_score(y_test, y_pred)

score = cross_val_score(history, X_test, y_test, cv=5, scoring='accuracy')
print(f"Cross Validation Score: {score.mean()}") # 0.79 % accuracy

print(f"Accuracy: {acc}") # 0.79 % accuracy